In [1]:
import numpy as np
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import os
import warnings
#import unicodedata
warnings.filterwarnings("ignore")
#from io import BytesIO
#import gzip
import requests

pd.set_option('display.max_rows',999)

In [2]:
import json

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## WB ES Surveys

In [4]:
es = pd.read_excel('files/ES.xlsx', sheet_name='Biggest Obstacle')

In [5]:
es = es.loc[(es['Subgroup'] == 'Location'),]
es.rename(columns={'Economy':'Country' , "Subgroup Level": "City"},inplace=True)

In [6]:
obstacles = [x for x in es.columns if 'biggest obstacle' in x]

whole = []
for country in es['Country'].unique():
    df1 = es[es['Country'] == country]
    for city in df1['City'].unique():
        df2 = df1[df1['City'] == city].set_index(['Country','City'])[obstacles]
        if df2.dropna().shape[0] != 0:
            stacked = df2.stack().sort_values(ascending = False).index.tolist()
            first = stacked[0][2]
            second = stacked[1][2]
            third = stacked[2][2]
            df2['first'] = first
            df2['second'] = second
            df2['third'] = third
            whole.append(df2)

all_ = obstacles + ['first', 'second', 'third']

es_obstacles = pd.concat(whole)[all_].reset_index()

es_obstacles = es_obstacles.set_index(['Country','City'])

KeyboardInterrupt: 

In [ ]:
top = ['first','second','third']

## Empezamos con *Doing Business*

Doing Busines

In [4]:
def descarga(api_key , url):
    headers = {'Ocp-Apim-Subscription-Key': '{0}'.format(api_key)}
    return requests.get(url, headers=headers).json()

In [5]:
key = '7c202aad75524b5a9c9f0a9fa42cbbbc'

countries = ['COL' , 'MEX' , 'PER' , 'DCP']

informes = ['DB_dwcp' , 'DB_sb' , 'DB_rp' , 'DB_ec']

url = 'https://wbgindicators.azure-api.net/DoingBusiness/api/GetSubnationalTopicOverView/{0}/{1}'

In [6]:
dwcp = pd.DataFrame()
sb = pd.DataFrame()
rp = pd.DataFrame()
ec = pd.DataFrame()

In [7]:
for e in countries:
    jsondata = descarga(key ,url.format('DB_dwcp',e))
    a = pd.DataFrame.from_dict(jsondata[list(jsondata.keys())[0]])
    a['Country'] = e
    dwcp = dwcp.append(a)

In [8]:
for e in countries:
    jsondata = descarga(key ,url.format('DB_sb',e))
    a = pd.DataFrame.from_dict(jsondata[list(jsondata.keys())[0]])
    a['Country'] = e
    sb = sb.append(a)

In [9]:
for e in countries:
    jsondata = descarga(key ,url.format('DB_rp',e))
    a = pd.DataFrame.from_dict(jsondata[list(jsondata.keys())[0]])
    a['Country'] = e
    rp = rp.append(a)

In [10]:
for e in ['MEX','PER']:
    jsondata = descarga(key ,url.format('DB_ec',e))
    a = pd.DataFrame.from_dict(jsondata[list(jsondata.keys())[0]])
    a['Country'] = e
    ec = ec.append(a)

Comenzmos con la limpieza de los datos

In [11]:
pais = {'COL' : 'Colombia' , 'MEX':'Mexico' , 'PER':'Peru'}

In [33]:
#Dealing With construction Permits: 'construcción'

construction = dwcp.loc[(dwcp['Country'].isin(['COL' , 'MEX' , 'PER'])),].replace({"Country": pais})
construction.rename(columns={'economyName':'City'},inplace=True)

construction.drop(columns=construction.columns.tolist()[1:6],inplace=True)
construction.drop(columns=construction.columns.tolist()[2],inplace=True)

construction.set_index(['Country','City'],inplace=True)

mylist = construction.columns.tolist()[1:]

for col in mylist:
    construction[col] = construction[col].astype('float')

mylist = construction.columns.tolist()[3:5]

for col in mylist:
    construction[col] = construction[col].astype('int')
    
construction.rename(columns={'dtf':'Construction_Permits_Score'},inplace=True)

In [34]:
construction.to_excel('C:/Users/Franco/Python/CAF/files/Doing Business/construction.xlsx')

In [35]:
#Starting Business: 'negocio'

negocio = sb.loc[(sb['Country'].isin(['COL' , 'MEX' , 'PER'])),].replace({"Country": pais})

negocio.drop(columns=negocio.columns.tolist()[1:6],inplace=True)
negocio.drop(columns=negocio.columns.tolist()[2],inplace=True)

negocio.rename(columns={'economyName':'City'},inplace=True)
negocio.set_index(['Country','City'],inplace=True)

mylist = negocio.columns.tolist()[1:]

for col in mylist:
    negocio[col] = negocio[col].astype('float')

mylist = negocio.columns.tolist()[3:5]

for col in mylist:
    negocio[col] = negocio[col].astype('int')

negocio.rename(columns={'dtf':'Starting_Business_Score'},inplace=True)

In [36]:
negocio.to_excel('C:/Users/Franco/Python/CAF/files/Doing Business/negocio.xlsx')

In [37]:
#Registering Property: 'propiedad'

propiedad = rp.loc[(rp['Country'].isin(['COL' , 'MEX' , 'PER'])),].replace({"Country": pais})

propiedad.drop(columns=propiedad.columns.tolist()[1:6],inplace=True)
propiedad.drop(columns=propiedad.columns.tolist()[2],inplace=True)

propiedad.rename(columns={'economyName':'City'},inplace=True)
propiedad.set_index(['Country','City'],inplace=True)

mylist = propiedad.columns.tolist()[1:]

for col in mylist:
    propiedad[col] = propiedad[col].astype('float')

mylist = propiedad.columns.tolist()[3:5]

for col in mylist:
    propiedad[col] = propiedad[col].astype('int')
    
propiedad.rename(columns={'dtf':'Registering_Property_Score'},inplace=True)

In [38]:
propiedad.to_excel('C:/Users/Franco/Python/CAF/files/Doing Business/propiedad.xlsx')

In [39]:
#Enforcing Contracts: 'contratos'

contratos = ec.loc[(ec['Country'].isin(['COL' , 'MEX' , 'PER'])),].replace({"Country": pais})

contratos.drop(columns=contratos.columns.tolist()[1:6],inplace=True)
contratos.drop(columns=contratos.columns.tolist()[2],inplace=True)

contratos.rename(columns={'economyName':'City'},inplace=True)
contratos.set_index(['Country','City'],inplace=True)

mylist = contratos.columns.tolist()[1:]

for col in mylist:
    contratos[col] = contratos[col].astype('float')

mylist = contratos.columns.tolist()[3:5]

for col in mylist:
    contratos[col] = contratos[col].astype('int')

contratos.rename(columns={'dtf':'Enforcing_Contracts_Score'},inplace=True)

In [40]:
contratos.to_excel('C:/Users/Franco/Python/CAF/files/Doing Business/contratos.xlsx')

In [361]:
DB = pd.DataFrame(construction['Construction_Permits_Score']).join(negocio['Starting_Business_Score'],how='inner').join(propiedad['Registering_Property_Score'],how='left').join(contratos['Enforcing_Contracts_Score'],how='left')

In [1]:
DB = DB.reset_index()
DB['City'] = DB['City'].str.lower().apply(lambda x: x.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')).str.title()
DB = DB.set_index(['Country','City'])
DB = DB.drop(columns='index')

NameError: name 'DB' is not defined

In [360]:
DB.to_excel('C:/Users/Franco/Python/CAF/files/Doing Business/doing_biz.xlsx')